# Day 65

Updating my rankings for teams based on total fantasy points allowed to the position through Week 16. I'll also run it for the past 4 weeks. Other updates:

1. Added new fields which give the *relevant* average number of touchdowns allowed to opponents for a position. For example, when looking at QBs, we only see the average *passing* touchdowns allowed by that opponent. For RBs, the field changes to the average *rushing* touchdowns.
2. Wrote a helper function, `print_output()` that should help with re-used code.

In [151]:
import pandas as pd
import sqlite3

# Create database connection
conn = sqlite3.connect('../../data/db/database_test.db')

In [152]:
season = 2022
week = 16

In [153]:
query = f"""
WITH fantasy_totals_by_team AS (
    -- Get the total fantasy points scored per team, per position, per week
    SELECT
        recent_team,
        position,
        week,
        SUM(passing_tds) AS tot_pass_tds,
        SUM(rushing_tds) AS tot_rush_tds,
        SUM(receiving_tds) AS tot_rec_tds,
        SUM(fantasy_points) AS tot_pts,
        SUM(fantasy_points_ppr) AS tot_pts_ppr
    FROM weekly
    WHERE season = {season}
        AND week <= {week}
        AND position in ('QB', 'RB', 'WR', 'TE')
    GROUP BY recent_team, position, week),
-- Get schedule info
    home_games AS (
        SELECT 
            game_id,
            season,
            week, 
            home_team AS team,
            away_team AS opp_team
        FROM schedules
        WHERE season = {season}
            AND week <= {week}
    ),
    away_games AS (
       SELECT 
            game_id,
            season,
            week, 
            away_team AS team,
            home_team AS opp_team
        FROM schedules
        WHERE season = {season}
            AND week <= {week}
    ),
    stacked AS (
        SELECT *
        FROM home_games
        UNION
        SELECT *
        FROM away_games
    ),
-- Join the two temp datasets
-- Every team will have it's total fantasy points per team appended to it
joined_data AS (
    SELECT
        game_id,
        season,
        stacked.week,
        team,
        opp_team,
        position,
        tot_pass_tds,
        tot_rush_tds,
        tot_rec_tds,
        tot_pts,
        tot_pts_ppr
    FROM stacked
    LEFT JOIN fantasy_totals_by_team
        ON fantasy_totals_by_team.recent_team = stacked.team
        AND fantasy_totals_by_team.week = stacked.week),
-- Only use opp_team to get the points scored scored against
total_pts_scored_against AS (
    SELECT
        opp_team,
        position,
        ROUND(AVG(tot_pass_tds), 2) AS avg_pass_tds,
        ROUND(AVG(tot_rush_tds), 2) AS avg_rush_tds,
        ROUND(AVG(tot_rec_tds), 2) AS avg_rec_tds,
        SUM(tot_pts) AS tot_pts_scored,
        SUM(tot_pts_ppr) AS tot_pts_ppr_scored
    FROM joined_data
    GROUP BY opp_team, position),
-- Utilize RANK() to get the season-level rankings
season_rankings AS (
    SELECT
        opp_team AS team,
        position,
        avg_pass_tds,
        avg_rush_tds,
        avg_rec_tds,
        DENSE_RANK() OVER(PARTITION BY position ORDER BY tot_pts_scored DESC) AS r_pts_scored,
        DENSE_RANK() OVER(PARTITION BY position ORDER BY tot_pts_ppr_scored DESC) AS r_pts_ppr_scored
    FROM total_pts_scored_against
    ORDER BY position, r_pts_ppr_scored)
SELECT *
FROM season_rankings --season_rankings
"""

df = pd.read_sql(query, conn)
df

,team,position,avg_pass_tds,avg_rush_tds,avg_rec_tds,r_pts_scored,r_pts_ppr_scored
0,DET,QB,1.60,0.47,0.00,1,1
1,MIA,QB,1.67,0.20,0.00,2,2
2,TEN,QB,1.73,0.20,0.00,3,3
3,KC,QB,2.07,0.13,0.00,4,4
4,MIN,QB,1.40,0.20,0.00,5,5
...,...,...,...,...,...,...,...
123,LAC,WR,0.00,0.00,0.87,28,28
124,HOU,WR,0.00,0.07,0.33,29,29
125,SEA,WR,0.00,0.00,0.73,30,30
126,NYJ,WR,0.00,0.00,0.67,31,31


## Define helpful objects and functions

In [154]:
# Set columns for given position
config = {
    'QB': ['team', 'position', 'avg_pass_tds', 'r_pts_scored', 'r_pts_ppr_scored'],
    'RB': ['team', 'position', 'avg_rush_tds', 'r_pts_scored', 'r_pts_ppr_scored'],
    'WR': ['team', 'position', 'avg_rec_tds', 'r_pts_scored', 'r_pts_ppr_scored'],
    'TE': ['team', 'position', 'avg_rec_tds', 'r_pts_scored', 'r_pts_ppr_scored']
}

In [155]:
def print_output(df, d, scoring='standard'):
    """
    Given dataframe, dictionary of position and fields to output, and scoring system,
    produce output tables. Default is standard scoring.
    """

    if scoring == 'ppr':
        for pos, cols in d.items():
        
            temp = df.query(f"(position == '{pos}') & (r_pts_ppr_scored <= 3 | r_pts_ppr_scored >= 30)")[cols]\
                    .drop('r_pts_scored', axis=1).sort_values('r_pts_ppr_scored')
            
            print(temp)
    
    else:
        for pos, cols in d.items():

                # top_3 = df.query(f"position == '{pos}'")[cols].sort_values('r_pts_scored')[:3]
                # bottom_3 = df.query(f"position == '{pos}'")[cols].sort_values('r_pts_scored')[:-3]
                # temp = pd.concat([top_3, bottom_3]).drop('r_pts_ppr_scored', axis=1)
            
                temp = df.query(f"(position == '{pos}') & (r_pts_scored <= 3 | r_pts_scored >= 30)")[cols]\
                    .drop('r_pts_ppr_scored', axis=1).sort_values('r_pts_scored')
            
                print(temp)
    
    return

## Standard Scoring through Week 16

In [156]:
print_output(df, config, 'standard')

   team position  avg_pass_tds  r_pts_scored
0   DET       QB          1.60             1
1   MIA       QB          1.67             2
2   TEN       QB          1.73             3
29   SF       QB          1.07            30
30  BUF       QB          1.20            31
31  DEN       QB          0.93            32
   team position  avg_rush_tds  r_pts_scored
32  HOU       RB          1.20             1
33  SEA       RB          0.87             2
35  CHI       RB          1.07             3
62   NE       RB          0.20            30
61  TEN       RB          0.33            31
63   SF       RB          0.33            32
    team position  avg_rec_tds  r_pts_scored
97   TEN       WR         1.27             1
98   DET       WR         0.93             2
96   MIN       WR         0.87             3
125  SEA       WR         0.73            30
126  NYJ       WR         0.67            31
127  DEN       WR         0.47            32
   team position  avg_rec_tds  r_pts_scored
65  SEA    

## PPR Scoring through Week 16

In [157]:
print_output(df, config, 'ppr')

   team position  avg_pass_tds  r_pts_ppr_scored
0   DET       QB          1.60                 1
1   MIA       QB          1.67                 2
2   TEN       QB          1.73                 3
29   SF       QB          1.07                30
30  BUF       QB          1.20                31
31  DEN       QB          0.93                32
   team position  avg_rush_tds  r_pts_ppr_scored
32  HOU       RB          1.20                 1
33  SEA       RB          0.87                 2
34  ARI       RB          0.80                 3
61  TEN       RB          0.33                30
62   NE       RB          0.20                31
63   SF       RB          0.33                32
    team position  avg_rec_tds  r_pts_ppr_scored
96   MIN       WR         0.87                 1
97   TEN       WR         1.27                 2
98   DET       WR         0.93                 3
125  SEA       WR         0.73                30
126  NYJ       WR         0.67                31
127  DEN       WR   

## Last 4 Weeks
To get the rankings for just the last 4 weeks I'll update the SQL query slightly, using `BETWEEN` in the `WHERE` clauses.

In [158]:
query = f"""
WITH fantasy_totals_by_team AS (
    -- Get the total fantasy points scored per team, per position, per week
    SELECT
        recent_team,
        position,
        week,
        SUM(passing_tds) AS tot_pass_tds,
        SUM(rushing_tds) AS tot_rush_tds,
        SUM(receiving_tds) AS tot_rec_tds,
        SUM(fantasy_points) as tot_pts,
        SUM(fantasy_points_ppr) as tot_pts_ppr
    FROM weekly
    WHERE season = {season}
        AND week BETWEEN {week - 3} AND {week}
        AND position in ('QB', 'RB', 'WR', 'TE')
    GROUP BY recent_team, position, week),
-- Get schedule info
    home_games AS (
        SELECT 
            game_id,
            season,
            week, 
            home_team AS team,
            away_team AS opp_team
        FROM schedules
        WHERE season = {season}
            AND week BETWEEN {week - 3} AND {week}
    ),
    away_games AS (
       SELECT 
            game_id,
            season,
            week, 
            away_team AS team,
            home_team AS opp_team
        FROM schedules
        WHERE season = {season}
            AND week BETWEEN {week - 3} AND {week}
    ),
    stacked AS (
        SELECT *
        FROM home_games
        UNION
        SELECT *
        FROM away_games
    ),
-- Join the two temp datasets
-- Every team will have it's total fantasy points per team appended to it
joined_data AS (
    SELECT
        game_id,
        season,
        stacked.week,
        team,
        opp_team,
        position,
        tot_pass_tds,
        tot_rush_tds,
        tot_rec_tds,
        tot_pts,
        tot_pts_ppr
    FROM stacked
    LEFT JOIN fantasy_totals_by_team
        ON fantasy_totals_by_team.recent_team = stacked.team
        AND fantasy_totals_by_team.week = stacked.week),
-- Only use opp_team to get the points scored scored against
total_pts_scored_against AS (
    SELECT
        opp_team,
        position,
        ROUND(AVG(tot_pass_tds), 2) AS avg_pass_tds,
        ROUND(AVG(tot_rush_tds), 2) AS avg_rush_tds,
        ROUND(AVG(tot_rec_tds), 2) AS avg_rec_tds,
        SUM(tot_pts) AS tot_pts_scored,
        SUM(tot_pts_ppr) AS tot_pts_ppr_scored
    FROM joined_data
    GROUP BY opp_team, position),
-- Utilize RANK() to get the season-level rankings
season_rankings AS (
    SELECT
        opp_team AS team,
        position,
        avg_pass_tds,
        avg_rush_tds,
        avg_rec_tds,
        DENSE_RANK() OVER(PARTITION BY position ORDER BY tot_pts_scored DESC) AS r_pts_scored,
        DENSE_RANK() OVER(PARTITION BY position ORDER BY tot_pts_ppr_scored DESC) AS r_pts_ppr_scored
    FROM total_pts_scored_against
    ORDER BY position, r_pts_ppr_scored)
SELECT *
FROM season_rankings
"""

df = pd.read_sql(query, conn)
df

,team,position,avg_pass_tds,avg_rush_tds,avg_rec_tds,r_pts_scored,r_pts_ppr_scored
0,PHI,QB,2.00,0.25,0.00,1,1
1,KC,QB,2.25,0.50,0.00,2,2
2,TEN,QB,1.75,0.50,0.00,3,3
3,NYG,QB,2.00,0.25,0.00,4,4
4,MIA,QB,2.00,0.00,0.00,5,5
...,...,...,...,...,...,...,...
123,DEN,WR,0.00,0.00,0.25,31,28
124,SEA,WR,0.00,0.00,0.50,28,29
125,GB,WR,0.00,0.00,0.33,27,30
126,NO,WR,0.00,0.00,0.00,32,31


## Standard Scoring Weeks 13-16

In [159]:
print_output(df, config, 'standard')

   team position  avg_pass_tds  r_pts_scored
0   PHI       QB          2.00             1
1    KC       QB          2.25             2
2   TEN       QB          1.75             3
29   GB       QB          0.67            30
30  CLE       QB          0.75            31
31  ARI       QB          0.67            32
   team position  avg_rush_tds  r_pts_scored
32  HOU       RB          1.00             1
33  SEA       RB          1.25             2
36  IND       RB          2.00             3
61   GB       RB          0.67            30
60   SF       RB          0.00            31
63  CAR       RB          0.67            32
    team position  avg_rec_tds  r_pts_scored
96   DAL       WR         2.25             1
97   MIN       WR         1.00             2
98   NYG       WR         1.50             3
127  ATL       WR         0.67            30
123  DEN       WR         0.25            31
126   NO       WR         0.00            32
   team position  avg_rec_tds  r_pts_scored
64  DEN    

## PPR Scoring Weeks 13-16

In [160]:
print_output(df, config, 'ppr')

   team position  avg_pass_tds  r_pts_ppr_scored
0   PHI       QB          2.00                 1
1    KC       QB          2.25                 2
2   TEN       QB          1.75                 3
29   GB       QB          0.67                30
30  CLE       QB          0.75                31
31  ARI       QB          0.67                32
   team position  avg_rush_tds  r_pts_ppr_scored
32  HOU       RB          1.00                 1
33  SEA       RB          1.25                 2
34  DEN       RB          1.00                 3
61   GB       RB          0.67                30
62  ATL       RB          0.00                31
63  CAR       RB          0.67                32
    team position  avg_rec_tds  r_pts_ppr_scored
96   DAL       WR         2.25                 1
97   MIN       WR         1.00                 2
98   NYG       WR         1.50                 3
125   GB       WR         0.33                30
126   NO       WR         0.00                31
127  ATL       WR   